# Calculadora de avance

In [2]:
import pandas as pd
import os
import pickle
import json
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools
from firebase_admin import credentials, firestore
import firebase_admin
import excel2img
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

## Datos del proyecto en análisis

In [3]:
proyecto="cabanaconde"
mes=4
anio=2025

## Datos de Firestore

In [4]:
# Ruta a tu archivo de clave privada descargado
cred = credentials.Certificate("../credentials_firestore.json")

# Inicializa sólo si no hay ninguna app por defecto
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)

# Obtiene una instancia de la base de datos Firestore
db = firestore.client()

In [5]:
data_mantenimiento=db.collection("rutinarios").document(proyecto).get()
if data_mantenimiento.exists:
    print(data_mantenimiento.to_dict())
else:
    print("No existe el documento.")

{'otros': {'tipo_superficie': 'afirmado', 'estado_conservacion_via': 'bueno'}, 'expediente': {'codigo_ruta': 'R0405106', 'monto_total': 81120, 'coordenadas': {'fin': {'hemisferio': 'S', 'y': 8272525.68, 'progresiva': 13000, 'zona_letra': 'L', 'x': 813907.99, 'datum': 'WGS84', 'altitud': 1975, 'zona': 18}, 'inicio': {'hemisferio': 'S', 'y': 8272658.46, 'progresiva': 0, 'zona_letra': 'L', 'x': 819845.05, 'datum': 'WGS84', 'altitud': 2877, 'zona': 18}}, 'nombre': 'MANTENIMIENTO VIAL RUTINARIO DEL CAMINO VECINAL EMP. R0405104 (CABANACONDE) A PTE. CABANACONDE - CHOCO (L.P. CASTILLA), DISTRITO DE CABANACONDE, PROVINCIA DE CAYLLOMA, REGIÓN AREQUIPA', 'longitud': 13000}, 'datos_generales': {'distritos': ['Cabanaconde'], 'region': 'Arequipa', 'provincia': 'Caylloma'}, 'other': {'tipo_superficie': 'afirmado', 'estado_conservacion_via': 'bueno'}, 'contrato': {'jefe_mantenimiento': {'titulo': 'Ingeniero', 'nombre': 'Genaro', 'dni': 0, 'apellido': 'Tinta Cáceres'}, 'numero_trabajadores': 4, 'denomi

In [6]:
progresiva_inicio=data_mantenimiento.to_dict()["expediente"]["coordenadas"]["inicio"]["progresiva"]
progresiva_fin=data_mantenimiento.to_dict()["expediente"]["coordenadas"]["fin"]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
13000


## Cargando cargas de trabajo programadas

In [7]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cargas_trabajo.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    df_cargas_trabajo_programadas = pickle.load(f)

df_cargas_trabajo_programadas.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR103,0.000000,0.000000,6.650000,6.650000,6.650000,6.650000,0.000000,6.650000,0.000000,33.25
MR104,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,0.000000,136.09
MR201,1515.151818,2272.727727,1893.939773,2272.727727,2272.727727,1893.939773,2272.727727,1893.939773,378.787955,16666.67
MR202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR203,42.424545,42.424545,84.849091,84.849091,42.424545,42.424545,42.424545,42.424545,42.424545,466.67
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR301,3606.515455,5409.773182,4508.144318,5409.773182,5409.773182,4508.144318,5409.773182,4508.144318,901.628864,39671.67


In [8]:
#eliminando el total
df_cargas_trabajo_programadas = df_cargas_trabajo_programadas.iloc[:-1]

In [9]:
# Concatenar la ruta completa al archivo .pkl
ruta_cronograma_anual = os.path.join("data",proyecto, f"{proyecto}_cronograma_anual.pkl")

# Leer el archivo pickle
with open(ruta_cronograma_anual, "rb") as f:
    df_cronograma_anual = pickle.load(f)

df_cronograma_anual.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
MR103,0,0,1,1,1,1,0,1,0,5
MR104,2,2,2,2,2,2,2,2,0,16
MR201,4,6,5,6,6,5,6,5,1,44
MR202,0,0,0,0,0,0,0,0,0,0
MR203,1,1,2,2,1,1,1,1,1,11
MR204,0,0,0,0,0,0,0,0,0,0
MR205,0,0,0,0,0,0,0,0,0,0
MR206,0,0,0,0,0,0,0,0,0,0
MR301,4,6,5,6,6,5,6,5,1,44
MR401,0,1,1,0,0,1,1,1,0,5


## Cargando actividades

In [10]:
ruta_actividades= os.path.join("data", "general_data", "actividades.json")
with open(ruta_actividades, 'r', encoding='utf-8') as archivo:
    actividades = json.load(archivo)
# Ahora 'datos' es un diccionario de Python
print(actividades)

[{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 3.6, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 360, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 9, 'unidad': 'm3'}}]}}, {'key': 'MR200', 'value': {'label': 'Limpieza de obras de arte', 'value': [{'key': 'MR201', 'value': {'label': 'Limpieza de cunetas', 'carga_trabajo': 1440, 'unidad': 'm'}}, {'key': 'MR202', 'value': {'label': 'Limpieza de alcantarillas', 'carga_trabajo': 0, 'unidad': 'unidad'}}, {'key': 'MR203', 'value': {'label': 'Limpieza de badén', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR204', 'value': {'label': 'Limpieza de zanjas de coronación', 'carga_trabajo': 0, 'unidad': 'm'}}, {'key': 'MR205', 'value': {'label': 'Limpieza de pontones', 'carga

## Cargando cargas de trabajo del contratista

In [11]:
ruta_cargas_trabajo= os.path.join("data", proyecto,str(mes), f"cargas_trabajo.json")
with open(ruta_cargas_trabajo, 'r', encoding='utf-8') as archivo:
    cargas_trabajo_contratista_presente_mes = json.load(archivo)
# Ahora 'cargas_trabajo_contratista' es un diccionario de Python
print(cargas_trabajo_contratista_presente_mes)

{'MR101': 3.6, 'MR102': 360, 'MR201': 1440, 'MR401': 10, 'MR104': 9}


## Funciones utiles

In [12]:
def calcular_porcentaje(cargas_mes, cargas_totales, carga_por_dia):
    """
    Devuelve el porcentaje del proyecto programado para este mes,
    ponderado por la carga de trabajo diaria de cada actividad.
    """
    total_dias_mes = 0.0
    total_dias_proyecto = 0.0

    for actividad, dias_diarios in carga_por_dia.items():
        # Ignoramos actividades sin capacidad diaria definida o totales cero
        if dias_diarios <= 0:
            continue

        carga_mes = cargas_mes.get(actividad, 0.0)
        carga_total = cargas_totales.get(actividad, 0.0)

        # Convertimos cargas a días
        total_dias_mes += carga_mes / dias_diarios
        total_dias_proyecto += carga_total / dias_diarios
        # print(
        #     {
        #         "actividad": actividad,
        #         "total_dias_mes": total_dias_mes,
        #         "total_dias_proyecto": total_dias_proyecto,
        #     }
        # )

    if total_dias_proyecto == 0:
        return 0.0

    return total_dias_mes / total_dias_proyecto

## Calculo

In [13]:
actividades=df_cargas_trabajo_programadas.index.to_list()
print(actividades)

['MR101', 'MR102', 'MR103', 'MR104', 'MR201', 'MR202', 'MR203', 'MR204', 'MR205', 'MR206', 'MR301', 'MR401', 'MR501', 'MR601', 'MR701', 'MR702']


### Avance programado vigente en el mes

In [14]:
# Buscamos la columna que termina en "-04"
col = [c for c in df_cargas_trabajo_programadas.columns if c.endswith(f"-{mes:02d}")][0]

# Extraemos la Serie y la pasamos a dict
cargas_trabajo_programdas_presente_mes = df_cargas_trabajo_programadas[col].to_dict()

print(cargas_trabajo_programdas_presente_mes)
# {'MR101': 1.540444, 'MR102': 153.14833, ... }

{'MR101': 1.5404444444444443, 'MR102': 153.1483333333333, 'MR103': 0.0, 'MR104': 17.01125, 'MR201': 1515.151818181818, 'MR202': 0.0, 'MR203': 42.42454545454546, 'MR204': 0.0, 'MR205': 0.0, 'MR206': 0.0, 'MR301': 3606.5154545454543, 'MR401': 0.0, 'MR501': 0.0, 'MR601': 0.0, 'MR701': 3.8659999999999997, 'MR702': 0.0}


In [15]:
# Opción A: usando df.columns
ultima_col = df_cargas_trabajo_programadas.columns[-1]
cargas_trabajo_programadas_totales = df_cargas_trabajo_programadas[ultima_col].to_dict()
print(cargas_trabajo_programadas_totales)

{'MR101': 17.329999999999995, 'MR102': 1837.78, 'MR103': 33.25, 'MR104': 136.09, 'MR201': 16666.67, 'MR202': 0.0, 'MR203': 466.67, 'MR204': 0.0, 'MR205': 0.0, 'MR206': 0.0, 'MR301': 39671.67, 'MR401': 29.33, 'MR501': 0.0, 'MR601': 112.67000000000002, 'MR701': 19.33, 'MR702': 0.0}


In [16]:
dias_totales_por_actividad=df_cronograma_anual[ultima_col].to_dict()
print(dias_totales_por_actividad)

{'MR101': 45, 'MR102': 60, 'MR103': 5, 'MR104': 16, 'MR201': 44, 'MR202': 0, 'MR203': 11, 'MR204': 0, 'MR205': 0, 'MR206': 0, 'MR301': 44, 'MR401': 5, 'MR501': 0, 'MR601': 5, 'MR701': 5, 'MR702': 0, 'TOTAL': 240}


In [17]:
carga_por_dia = {
    k: (
        (cargas_trabajo_programadas_totales.get(k, 0) / dias_totales_por_actividad[k])
        if dias_totales_por_actividad[k] != 0
        else 0
    )
    for k in dias_totales_por_actividad
}

print(carga_por_dia)

{'MR101': 0.385111111111111, 'MR102': 30.629666666666665, 'MR103': 6.65, 'MR104': 8.505625, 'MR201': 378.7879545454545, 'MR202': 0, 'MR203': 42.42454545454546, 'MR204': 0, 'MR205': 0, 'MR206': 0, 'MR301': 901.6288636363636, 'MR401': 5.866, 'MR501': 0, 'MR601': 22.534000000000002, 'MR701': 3.8659999999999997, 'MR702': 0, 'TOTAL': 0.0}


In [18]:
porcentajes_programado_vigente_mes = {
    k: (cargas_trabajo_programdas_presente_mes.get(k, 0) / cargas_trabajo_programadas_totales[k]
        if cargas_trabajo_programadas_totales[k] != 0 else 0)
    for k in cargas_trabajo_programadas_totales
}
print(porcentajes_programado_vigente_mes)

{'MR101': 0.0888888888888889, 'MR102': 0.08333333333333333, 'MR103': 0.0, 'MR104': 0.125, 'MR201': 0.09090909090909091, 'MR202': 0, 'MR203': 0.09090909090909091, 'MR204': 0, 'MR205': 0, 'MR206': 0, 'MR301': 0.09090909090909091, 'MR401': 0.0, 'MR501': 0, 'MR601': 0.0, 'MR701': 0.2, 'MR702': 0}


In [19]:
porcentaje_programado_vigente_mes=calcular_porcentaje(cargas_mes=cargas_trabajo_programdas_presente_mes,cargas_totales=cargas_trabajo_programadas_totales,carga_por_dia=carga_por_dia)
print(porcentaje_programado_vigente_mes)

0.0875


#### Como no calcular

In [20]:
"""
En este script calculamos el avance mensual de dos formas:

1. Avance ponderado por carga de trabajo:
   - Cada actividad tiene un porcentaje de ejecución y una 'carga_por_dia' (peso).
   - El avance global se obtiene como la suma de (porcentaje_ejecutado * carga_por_dia) 
     dividida entre la suma total de todas las cargas.
   - De esta forma, las actividades que representan más trabajo diario (mayor carga)
     aportan proporcionalmente más al avance general.

2. Promedio simple de porcentajes:
   - Se suman todos los porcentajes de ejecución y se dividen entre el número de actividades.
   - Este cálculo asume que todas las actividades tienen la misma importancia, 
     independientemente de su carga real de trabajo.
   - Es incorrecto porque trata igual un metro de vereda (baja carga) que un kilómetro de carretera (alta carga),
     y podría dar un avance engañoso (por ejemplo, 100% de vereda + 0% de carretera → 50%).
   
Por eso, el método ponderado refleja fielmente el progreso real,
mientras que el promedio simple solo sirve para comparaciones o ilustrar el sesgo del cálculo no ponderado.
"""

"\nEn este script calculamos el avance mensual de dos formas:\n\n1. Avance ponderado por carga de trabajo:\n   - Cada actividad tiene un porcentaje de ejecución y una 'carga_por_dia' (peso).\n   - El avance global se obtiene como la suma de (porcentaje_ejecutado * carga_por_dia) \n     dividida entre la suma total de todas las cargas.\n   - De esta forma, las actividades que representan más trabajo diario (mayor carga)\n     aportan proporcionalmente más al avance general.\n\n2. Promedio simple de porcentajes:\n   - Se suman todos los porcentajes de ejecución y se dividen entre el número de actividades.\n   - Este cálculo asume que todas las actividades tienen la misma importancia, \n     independientemente de su carga real de trabajo.\n   - Es incorrecto porque trata igual un metro de vereda (baja carga) que un kilómetro de carretera (alta carga),\n     y podría dar un avance engañoso (por ejemplo, 100% de vereda + 0% de carretera → 50%).\n\nPor eso, el método ponderado refleja fielme

In [21]:
def calcular_promedio_avance_mal(porcentajes):
    """
    Devuelve el promedio simple de avance.
    - porcentajes: dict actividad -> porcentaje ejecutado (0..1)
    """
    total_actividades = len(porcentajes)
    if total_actividades == 0:
        return 0.0
    suma_porcentajes = sum(porcentajes.values())
    return suma_porcentajes / total_actividades

In [22]:
print(calcular_promedio_avance_mal(porcentajes=porcentajes_programado_vigente_mes)) # Este metodo esta mal, porque solo promedia los avances

0.048121843434343434


## Avance ejecutado en el mes

In [23]:
porcentajes_ejecutados_vigente_mes = {
    k: (cargas_trabajo_contratista_presente_mes.get(k, 0) / cargas_trabajo_programadas_totales[k]
        if cargas_trabajo_contratista_presente_mes.get(k, 0)  != 0 else 0)
    for k in cargas_trabajo_programadas_totales
}
print(porcentajes_ejecutados_vigente_mes)

{'MR101': 0.20773225620311606, 'MR102': 0.19588851766805604, 'MR103': 0, 'MR104': 0.06613270629730325, 'MR201': 0.08639998272000346, 'MR202': 0, 'MR203': 0, 'MR204': 0, 'MR205': 0, 'MR206': 0, 'MR301': 0, 'MR401': 0.34094783498124787, 'MR501': 0, 'MR601': 0, 'MR701': 0, 'MR702': 0}


In [24]:
print(cargas_trabajo_contratista_presente_mes)

{'MR101': 3.6, 'MR102': 360, 'MR201': 1440, 'MR401': 10, 'MR104': 9}


In [25]:
porcentaje_ejecutado_vigente_mes=calcular_porcentaje(cargas_mes=cargas_trabajo_contratista_presente_mes,cargas_totales=cargas_trabajo_programadas_totales,carga_por_dia=carga_por_dia)
print(porcentaje_ejecutado_vigente_mes)

0.11527385126902844


In [ ]:
data_to_see = {
    "porcentaje_programado_vigente_mes": porcentaje_programado_vigente_mes * 100,
    "porcentaje_ejecutado_vigente_mes": porcentaje_ejecutado_vigente_mes * 100,
    "porcentaje_programado_acumulado_vigente_mes": porcentaje_programado_vigente_mes
    * 100,
    "porcentaje_ejecutado_acumulado_vigente_mes": porcentaje_ejecutado_vigente_mes
    * 100,
}

In [ ]:
data_to_print = {
    "porcentaje_programado_vigente_mes": porcentaje_programado_vigente_mes,
    "porcentaje_ejecutado_vigente_mes": porcentaje_ejecutado_vigente_mes,
    "porcentaje_programado_acumulado_vigente_mes": porcentaje_programado_vigente_mes,
    "porcentaje_ejecutado_acumulado_vigente_mes": porcentaje_ejecutado_vigente_mes,
}

In [27]:
ruta_directorio = os.path.join("output", proyecto, str(mes))
ruta_archivo = os.path.join(ruta_directorio, "cargas_trabajo.json")

# Crear el directorio si no existe
os.makedirs(ruta_directorio, exist_ok=True)
# Guardar el diccionario en un archivo JSON

with open(ruta_archivo, "w", encoding="utf-8") as archivo_json:
    json.dump(data, archivo_json, ensure_ascii=False, indent=4)
